# Train

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

import sys
sys.path.append(os.path.abspath('..'))

from fiat.arch.image_seg import unet, resunet_arch
# from fiat.components.arch import 
from fiat.os import count
from fiat.data import TFR
from fiat.train import train
from tools.data import feature_dict, decode_img_seg, seg_train_gen
from fiat.DataAugment import flip_up_down, flip_left_right

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [2]:
csv_path = os.path.join('..', 'data', 'train.csv')
train_path = os.path.join('..', 'data', 'train_images')
tfr_path = os.path.join('..', 'tmp', 'TFRecords', 'train2')
ckpt_path = os.path.join('..', 'tmp', 'ckpt')

In [3]:
tfr = TFR(path=tfr_path,
          count=count(train_path),
          feature_dict=feature_dict, 
          shards=30, 
          compression='GZIP',
          c_level=1)

In [4]:
#gen = seg_train_gen(csv_path, train_path, sep='[_,]', nclass=4)
#tfr.write(gen)

In [7]:
arch = resunet_arch(reslayer='seresnext',
                    numlayers='18',
                    numstages=3,
                    channels=16,
                    n_class=4,
                    padding='SAME',
                    rate=0.25,
                    active='sigmoid')

read = tfr.read(decode_raw=decode_img_seg,
                split=10,
                valid=0, 
                augment=[flip_up_down(), flip_left_right()],
                buffer_size=100,
                num_parallel_reads=3)

info = train(arch, read, 
             loss='dice', metric='dice', optimizer='momentun',
             rate=1e-6, epoch=1, batch_size=1, early_stopping=1,
             verbose=2, retrain=False, reshape=[32, 200], reshape_method=3, ckpt_path=ckpt_path)

W1018 20:52:06.472469 140422773372736 deprecation.py:323] From /home/leechh/code/Severstal/fiat/train.py:74: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
W1018 20:52:06.484179 140422773372736 deprecation_wrapper.py:119] From /home/leechh/code/Severstal/fiat/components/arch/variable.py:24: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W1018 20:52:06.503931 140422773372736 deprecation.py:323] From /home/leechh/code/Severstal/fiat/components/arch/layers.py:20: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for upda

KeyboardInterrupt: 

In [ ]:
arch = unet(num_layers=4, 
            feature_growth_rate=16,
            n_class=4,
            channels=3,
            padding='SAME',
            dropout_rate=0.25,
            active='sigmoid')

read = tfr.read(decode_raw=decode_img_seg,
         split=10,
         valid=0,
         buffer_size=100,
         num_parallel_reads=2)

info2 = train(arch, read, 
             loss='dice', metric='dice', optimizer='momentun',
             rate=1e-6, epoch=1, batch_size=1, early_stopping=1,
             verbose=2, retrain=info, reshape=[32, 200], reshape_method=3,
              ckpt_path=ckpt_path, distrainable=['3', '4'])

In [ ]:
tf.all_variables()

In [ ]:
[i.name.split('/')[0].split('_')[-1] for i in tf.all_variables()]